# A simple univariate example

In [1]:
using DynamicPolynomials, MomentTools

┌ Info: Precompiling MomentTools [65cd4d85-9fe5-4fdc-93e2-c8798c615752]
└ @ Base loading.jl:1273
┌ Warning: Package MomentTools does not have Dualization in its dependencies:
│ - If you have MomentTools checked out for development and have
│   added Dualization as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MomentTools
└ Loading Dualization into MomentTools from project dependency, future warnings for MomentTools are suppressed.


In [2]:
X = @polyvar x
p = x^4+x^3-x-1

x⁴ + x³ - x - 1

We minimize $x^2$ under the constraint $p=0$. 

In [3]:
using JuMP, MosekTools; optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
v, M = minimize(x^2, [p], [], X, 4, optimizer)

(0.9999999936153254, 
A MOMent program with:
A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Dual model with Mosek attached
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, variables, y)

We use Mosek optimizer to solve the underlying SDP optimization problem. The first argument $x^2$ of the function `minimize` is the objective function. The second argument is the array of equality constraints, the third is the array of non-negativity constraints (here empty).

The function `minimize` returns `v` the optimal value found (here it is $0$ since we optimize "$0$") and `M` the Moment Model built for the optimization (which type is `MomentTools.MOM.Model`). 

The minimizers can be obtained from `M` as follows:

In [4]:
getminimizers(M)

1×2 Array{Float64,2}:
 -1.0  1.0

The minimizers are presented as a $n\times r$ matrix, where $n$ is the number of variables (here $n=1$) and $r$ is the number of points (here $r=2$). 

An approximation  of the associate measure can be obtained as follows:

In [5]:
getmeasure(M)

([0.515012473322808, 0.484987522139862], [-1.0000000010949475 0.9999999995824738])

It corresponds to the measure $0.5\, \delta_{-1} + 0.5\, \delta_{1}$ where $\delta_x$ is the Dirac measure supported on $x$.


Next, we search the root(s) with $x \geq 0$. For that purpose, we minimize `nothing` (i.e. $1$) and add the non-negativity constraint $x\geq 0$:

In [6]:
v, M = minimize(nothing, [p], [x], X, 4, optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 1.0000000000498959

Now we search the negative root with maximal value of $x$.

In [7]:
v, M = maximize(x, [p], [-x], X, 4, optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 -0.9999999994227465